## Make a Staging Dataset

In [1]:
dataset_id = "sabermetrics_staging"

In [2]:
!bq --location=US mk --dataset {dataset_id}

BigQuery error in mk operation: Dataset
'extended-ascent-252921:sabermetrics_staging' already exists.


## Import dataset from cloud storage

In [4]:
!bq --location=US load --autodetect --skip_leading_rows=1 \
--source_format=CSV {dataset_id}.jBatters \
'gs://sabermetrics_bucket_ajn873/raw/npbb.csv'

Waiting on bqjob_r6a6c6a7ff6e2ce51_0000016e71bc1c9a_1 ... (1s) Current status: DONE   


In [5]:
%%bigquery 
select count(*) from sabermetrics_staging.jBatters

,f0_
0,5234


## Make a Modeled Dataset

In [6]:
dataset_id = "sabermetrics_modeled"

In [7]:
!bq --location=US mk --dataset {dataset_id}

Dataset 'extended-ascent-252921:sabermetrics_modeled' successfully created.


In [21]:
# DON'T RUN AGAIN
%%bigquery
create or replace table sabermetrics_modeled.jBat as
select distinct row_number() over (partition BY Year) as index, Year, City, Team, TeamID, LName, FName, 
safe_cast(Avg as FLOAT64) as Avg, safe_cast(SLG as FLOAT64) as SLG, 
safe_cast(OBP as FLOAT64) as OBP, safe_cast(RiSP as FLOAT64) as RiSP, 
G, AB, R, H, _2B, _3B, HR, RBI, BB, HBP, SO, SB, GIDP, safe_cast(PA as INT64) as PA, 
Sac, E, PlayerID, Uniform, safe_cast(ARISP as FLOAT64) as ARISP, 
safe_cast(SF as INT64) as SF, safe_cast(SH as INT64) as SH, 
safe_cast(DP as INT64) as DP, Bats, LG, Throws, safe_cast(RC_27 as FLOAT64) as RC_27, 
safe_cast(A as INT64) as A, safe_cast(Pos1 as INT64) as Pos1, Pos,  
safe_cast(D_G as INT64) as D_G , safe_cast( PO as INT64) as PO , 
safe_cast(CS as INT64) as CS 
from sabermetrics_staging.jBatters
order by Year  

""


In [22]:
%%bigquery
UPDATE sabermetrics_modeled.jBat
SET playerpk = concat(cast(LName as string), cast(FName as string), cast(Team as string), cast(Year as string), cast(index as string))
WHERE FName is not null

""


In [23]:
%%bigquery
UPDATE sabermetrics_modeled.jBat
SET playerpk = concat(cast(LName as string), cast(Team as string), cast(Year as string), cast(index as string))
WHERE FName is null

""


In [26]:
%%bigquery
UPDATE sabermetrics_modeled.jBat
SET playerpk = concat(cast(FName as string), cast(Team as string), cast(Year as string), cast(index as string))
WHERE LName is null

""


In [27]:
%%bigquery
select count(distinct playerpk)
from sabermetrics_modeled.jBat

,f0_
0,5234


In [28]:
%%bigquery
select count(*)
from sabermetrics_modeled.jBat

,f0_
0,5234


## Cross Queries

In [33]:
%%bigquery
select pyspID, b.FName, b.LName, a.b_g, b.g
from seanlahman_modeled.player_stats_all a join sabermetrics_modeled.jBat b
on (a.yearID = b.Year and a.b_g = b.g)
where a.b_g > 100 limit 10

,pyspID,FName,LName,b_g,g
0,boggswa01199713B,Makato,Shiozaki,104,104
1,boggswa0119971P,Makato,Shiozaki,104,104
2,cangejo0119971OF,Shuji,Nishiyama,103,103
3,cangejo0119971OF,Koujirou,Machida,103,103
4,cangejo0119971P,Shuji,Nishiyama,103,103
5,cangejo0119971P,Koujirou,Machida,103,103
6,finlest0120011OF,Matsui,Kazuo,140,140
7,finlest0120011OF,Matsui,Hideki,140,140
8,finlest0120011OF,Takahashi,Yoshinobu,140,140
9,finlest0120011OF,Nishi,Toshihisa,140,140


In [34]:
%%bigquery
select a.yearID, a.teamID, avg (a.b_R) as average_team_runs 
from seanlahman_modeled.player_stats_all a
group by a.yearID, a.teamID
union distinct
select b.Year, b.Team, avg (b.R) as average_team_runs
from sabermetrics_modeled.jBat b
group by b.Year, b.Team
order by average_team_runs desc limit 10

,yearID,teamID,average_team_runs
0,1993,Fighters,44.833333
1,1993,Swallows,42.500000
2,1985,Tigers,41.411765
3,1996,Carp,38.823529
4,1994,Buffaloes,38.666667
5,1993,Carp,36.500000
6,1999,BayStars,36.052632
7,1976,Giants,35.529412
8,1993,Dragons,35.266667
9,1993,Buffaloes,34.500000


In [39]:
%%bigquery
select distinct pysID, a.yearID, b.FName, b.LName, a.b_SB, b.SB
from seanlahman_modeled.player_stats_all a join sabermetrics_modeled.jBat b
on (a.yearID = b.Year and a.b_SB = b.SB)
where a.b_SB > 5 and b.FName is not null and b.LName is not null 
order by b.SB desc

,pysID,yearID,FName,LName,b_SB,SB
0,cangejo0119951,1995,Kazuo,Matsui k,21,21
1,relafde0120011,2001,Shimizu,Masaji,13,13
2,relafde0120011,2001,Araki,Masahiro,13,13
3,finlest0120011,2001,Miyamoto,Shinya,11,11
4,finlest0120011,2001,Saeki,Takahiro,11,11
5,finlest0120011,2001,Morimoto,Hichori,11,11
6,pecotbi0119921,1992,Kenichi,Yamazaki,9,9
7,pecotbi0119921,1992,Chihiro,Hamana,9,9
8,martida0119951,1995,Norio,Tanabe,8,8
9,martida0119951,1995,Takahiro,Ikeyama,8,8
